In [1]:
import torch
import torch.nn as nn
import random
import numpy as np
import pandas as pd
import json
import cv2
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from functools import partial
from dataclasses import dataclass
from collections import OrderedDict
import torchvision
from IPython.display import clear_output
import matplotlib.pyplot as plt
import tqdm
from PIL import ImageFilter, Image
import torchvision.models as models
import pretrainedmodels as ptmodels
import torch.nn as nn
import torch.nn.functional as F
import math
#import torch.utils.model_zoo as model_zoo
from torch.nn import init

from network.meso import Meso4, MesoInception4
from network.models import model_selection

#from ff import model_selection
from pipeline.metrics import accuracy_b, log_loss
from pipeline.model_methods import validate_img, train_img, validate_vid, train_vid
from pipeline.data_loaders import load_img_dataset, load_img_val_dataset, strong_aug
from pipeline.image_extracting import extract_faces, FastMTCNN

from pipeline.metrics import accuracy_sigmoid, accuracy_sigmoid_mean, log_loss_sigmoid, log_loss_b, accuracy_b_mean
from pipeline.model_methods import validate_vid_bf, train_vid_bf
from pipeline.image_extracting import extract_faces, FastMTCNN, InceptionResnetV1, extract_faces_dlib, MTCNN
from pipeline.blazeface import BlazeFace
from efficientnet_pytorch import EfficientNet


random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
PATH = "model.h5"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
fast_mtcnn = FastMTCNN(
    stride=4,
    resize=1,
    margin=14,
    factor=0.9,
    #min_face_size=60,
    #keep_all=True,
    select_largest=True,
    device=device
)
transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
facedet = BlazeFace().to(device)
facedet.load_weights("pipeline/blazeface.pth")
facedet.load_anchors("pipeline/anchors.npy")
_ = facedet.train(False)

from pipeline.helpers.read_video_1 import VideoReader
from pipeline.helpers.face_extract_1 import FaceExtractor

frames_per_video = 17

video_reader = VideoReader()
video_read_fn = lambda x: video_reader.read_frames(x, num_frames=frames_per_video)
face_extractor = FaceExtractor(video_read_fn, facedet)

In [4]:
y_train = pd.read_csv(r'D:\Machine Learning\deepfake-detection\data\metadata.csv')
len(y_train[y_train.label == 1]), len(y_train[y_train.label == 0])

(100000, 19154)

In [5]:
#%%script false

batch_size = 16
normalize = torchvision.transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))

x_train = load_img_dataset(r'data\train', batch_size, resize=224, normalize=normalize)
x_val = load_img_val_dataset('data\img_val', 10, resize=224, normalize=normalize)

x_train_vid = r'D:\Machine Learning\deepfake-detection\data\train_set'
x_val_vid = r'data\val'
x_test_vid = r'D:\Machine Learning\deepfake-detection\data\cross_val'

In [6]:
class MyResNeXt(models.resnet.ResNet):
    def __init__(self, training=True):
        super(MyResNeXt, self).__init__(block=models.resnet.Bottleneck,
                                        layers=[3, 4, 6, 3], 
                                        groups=32, 
                                        width_per_group=4)
        #self.fc = nn.Linear(2048, 1)

In [7]:
model = EfficientNet.from_pretrained('efficientnet-b2')#models.resnext50_32x4d(pretrained=True) models.resnext101_32x8d(pretrained=True) MyResNeXt()
#model = models.resnet18(pretrained=True)
#model._fc = nn.Linear(1280, 1)
model.fc = nn.Linear(model.fc.in_features, 1)
'''
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(2048, 1)
)
'''

'\nmodel.fc = nn.Sequential(\n    nn.Dropout(0.5),\n    nn.Linear(2048, 1)\n)\n'

In [8]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [9]:
%%script false

#model.load_state_dict(torch.load("pretrained/resnext50_32x4d-7cdf4587.pth", map_location=device))
model.load_state_dict(torch.load("0.8134302675729158 0.40084000225509364.pth", map_location=device))
_ = model.eval()

Couldn't find program: 'false'


In [10]:
model._fc = nn.Linear(model._fc.in_features, 1)

'''
model.fc = nn.Sequential(
                nn.Linear(2048, 256), 
                nn.ReLU(), 
                nn.Dropout(0.5),
                nn.Linear(256, 1))
'''

'\nmodel.fc = nn.Sequential(\n                nn.Linear(2048, 256), \n                nn.ReLU(), \n                nn.Dropout(0.5),\n                nn.Linear(256, 1))\n'

In [11]:
model = model.to(device)

In [12]:
loss =  torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.) 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)
#scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda epoch: epoch // 30, lambda epoch: 0.95 ** epoch], last_epoch=-1)

In [13]:
for child in model.named_parameters():  
    print(child)

('conv1.weight', Parameter containing:
tensor([[[[-1.0419e-02, -6.1356e-03, -1.8098e-03,  ...,  5.6615e-02,
            1.7083e-02, -1.2694e-02],
          [ 1.1083e-02,  9.5276e-03, -1.0993e-01,  ..., -2.7124e-01,
           -1.2907e-01,  3.7424e-03],
          [-6.9434e-03,  5.9089e-02,  2.9548e-01,  ...,  5.1972e-01,
            2.5632e-01,  6.3573e-02],
          ...,
          [-2.7535e-02,  1.6045e-02,  7.2595e-02,  ..., -3.3285e-01,
           -4.2058e-01, -2.5781e-01],
          [ 3.0613e-02,  4.0960e-02,  6.2850e-02,  ...,  4.1384e-01,
            3.9359e-01,  1.6606e-01],
          [-1.3736e-02, -3.6746e-03, -2.4084e-02,  ..., -1.5070e-01,
           -8.2230e-02, -5.7828e-03]],

         [[-1.1397e-02, -2.6619e-02, -3.4641e-02,  ...,  3.2521e-02,
            6.6221e-04, -2.5743e-02],
          [ 4.5687e-02,  3.3603e-02, -1.0453e-01,  ..., -3.1253e-01,
           -1.6051e-01, -1.2826e-03],
          [-8.3730e-04,  9.8420e-02,  4.0210e-01,  ...,  7.0789e-01,
            3.6887e

       requires_grad=True))
('bn1.bias', Parameter containing:
tensor([ 2.3072e-01,  2.5382e-01, -1.0543e-06, -6.6439e-01, -1.6571e-08,
         1.6152e-01,  4.5450e-01, -4.3020e-07,  3.0051e-01, -8.0052e-06,
         3.4942e-01,  3.1148e-01, -2.4953e-01, -3.4749e-05,  1.0773e-01,
         2.1897e-01,  3.8141e-01, -5.2988e-01, -6.2864e-01,  5.7140e-01,
         2.9985e-01,  5.8430e-01,  4.8202e-01,  3.2853e-01,  1.9672e-01,
         1.9496e-01,  1.5215e-01,  8.5522e-02,  5.1314e-01,  1.5237e-02,
         1.6644e-01,  3.3239e-01,  2.4921e-01,  4.4337e-01, -2.8017e-01,
        -2.0385e-02, -2.4507e-07,  3.2134e-01, -4.9152e-08,  2.3777e-01,
         2.3291e-01,  3.1527e-01,  4.2776e-01,  2.9313e-01,  2.6379e-01,
         6.7598e-01,  4.2910e-01,  3.4566e-01, -8.6909e-08,  2.4729e-01,
         3.0316e-01,  6.1577e-01,  3.9835e-01,  3.3207e-01, -4.1219e-01,
         3.7807e-01,  1.7895e-01,  2.5748e-01, -4.4908e-01,  2.1306e-01,
         5.6934e-01,  5.7274e-01, -4.0238e-01,  2.3406e-01], 

('layer1.1.bn1.weight', Parameter containing:
tensor([0.3910, 0.4375, 0.3746, 0.3990, 0.3404, 0.3503, 0.2618, 0.2707, 0.2865,
        0.4308, 0.1895, 0.3041, 0.3837, 0.2944, 0.2105, 0.3304, 0.2943, 0.2887,
        0.2060, 0.4627, 0.2335, 0.1831, 0.4489, 0.2830, 0.3389, 0.2997, 0.3503,
        0.2735, 0.3908, 0.2817, 0.2636, 0.4462, 0.3282, 0.3776, 0.4471, 0.3878,
        0.2516, 0.3172, 0.3661, 0.3166, 0.3818, 0.3128, 0.2274, 0.3627, 0.2902,
        0.2381, 0.2988, 0.2469, 0.3840, 0.2886, 0.3197, 0.2879, 0.3218, 0.4559,
        0.3500, 0.2420, 0.3396, 0.3519, 0.3839, 0.3806, 0.4039, 0.2826, 0.4594,
        0.3342], device='cuda:0', requires_grad=True))
('layer1.1.bn1.bias', Parameter containing:
tensor([-0.0997, -0.4755, -0.0474, -0.2698, -0.0834, -0.0072,  0.0474,  0.1022,
        -0.0170, -0.1471,  0.2307,  0.1447, -0.1775,  0.0273,  0.1559, -0.1836,
         0.1238, -0.1522,  0.0554, -0.2881, -0.2606,  0.2316, -0.3242, -0.0219,
        -0.2645,  0.0576, -0.2465,  0.0481, -0.3530,  0

('layer2.0.bn2.weight', Parameter containing:
tensor([0.1454, 0.3270, 0.3113, 0.2538, 0.4086, 0.3937, 0.4400, 0.3108, 0.3406,
        0.2168, 0.2170, 0.3857, 0.1971, 0.2692, 0.1663, 0.2454, 0.3232, 0.3686,
        0.3893, 0.3264, 0.3875, 0.4707, 0.1958, 0.4717, 0.1673, 0.3938, 0.3044,
        0.1929, 0.2175, 0.2119, 0.4230, 0.3683, 0.2455, 0.2229, 0.3370, 0.3229,
        0.2688, 0.3557, 0.2581, 0.4031, 0.4492, 0.3642, 0.2599, 0.1881, 0.1359,
        0.2958, 0.1913, 0.3065, 0.3981, 0.4102, 0.1874, 0.4516, 0.3340, 0.1628,
        0.3599, 0.1624, 0.2886, 0.1358, 0.4491, 0.2694, 0.4823, 0.3393, 0.4764,
        0.3155, 0.6005, 0.4654, 0.5264, 0.2991, 0.2992, 0.4621, 0.2614, 0.4247,
        0.4662, 0.4249, 0.3345, 0.2655, 0.4048, 0.3605, 0.1782, 0.3833, 0.2823,
        0.3843, 0.3307, 0.2151, 0.3317, 0.1458, 0.2771, 0.4917, 0.3199, 0.4222,
        0.1559, 0.4884, 0.3267, 0.3440, 0.1608, 0.4855, 0.2677, 0.1616, 0.3221,
        0.4243, 0.3661, 0.1893, 0.3400, 0.3648, 0.1779, 0.3544, 0.2852, 0.

        0.1326, 0.3915], device='cuda:0', requires_grad=True))
('layer2.1.bn2.bias', Parameter containing:
tensor([-0.1403, -0.0889, -0.4147, -0.2264, -0.0737, -0.3534, -0.3379, -0.0752,
        -0.1791,  0.0448, -0.2842, -0.1765, -0.1591, -0.0675, -0.1543, -0.1061,
        -0.2334, -0.0981, -0.0908, -0.0567, -0.1908, -0.2055, -0.2704, -0.1883,
        -0.3570, -0.1125, -0.1632, -0.0211, -0.1687, -0.2124, -0.1713, -0.0872,
        -0.2194, -0.1888, -0.2954, -0.4570, -0.0226, -0.0527,  0.0406, -0.0609,
        -0.0456, -0.1176, -0.0145,  0.0318, -0.2046, -0.0953, -0.0496, -0.1051,
        -0.0793, -0.1933, -0.1467, -0.3215, -0.3257, -0.2287, -0.0356, -0.1869,
        -0.1932, -0.0771,  0.2768, -0.0656, -0.0895, -0.2548, -0.2365,  0.0021,
        -0.0987, -0.3178,  0.1613,  0.0006, -0.2347, -0.4150, -0.1310, -0.3142,
        -0.2582, -0.5400,  0.0772, -0.2546, -0.4454, -0.0262, -0.0937, -0.2201,
        -0.2044, -0.0155, -0.0893, -0.2167,  0.1112, -0.0619, -0.1217, -0.1593,
        -0.13

...

('layer4.0.bn1.weight', Parameter containing:
tensor([0.2427, 0.2232, 0.2511, 0.2288, 0.2074, 0.2905, 0.2482, 0.3102, 0.2749,
        0.2892, 0.2448, 0.1759, 0.2426, 0.2780, 0.2315, 0.2631, 0.3383, 0.2785,
        0.2536, 0.2989, 0.2335, 0.2812, 0.3486, 0.2778, 0.2280, 0.2547, 0.3032,
        0.2468, 0.2512, 0.2973, 0.2577, 0.3200, 0.2385, 0.2714, 0.2532, 0.2625,
        0.3344, 0.2626, 0.1838, 0.2839, 0.2187, 0.2666, 0.2858, 0.2471, 0.2915,
        0.2332, 0.2637, 0.2691, 0.2432, 0.2384, 0.2356, 0.2525, 0.2564, 0.2451,
        0.2529, 0.2522, 0.2800, 0.3165, 0.2340, 0.2634, 0.2569, 0.1942, 0.2621,
        0.2205, 0.2301, 0.2323, 0.2811, 0.1897, 0.2280, 0.3472, 0.2717, 0.3191,
        0.2440, 0.2719, 0.2781, 0.2262, 0.3444, 0.2648, 0.2725, 0.2851, 0.2039,
        0.2935, 0.2742, 0.2774, 0.2654, 0.2430, 0.2721, 0.2708, 0.3085, 0.2895,
        0.2596, 0.2147, 0.3119, 0.3449, 0.2262, 0.2814, 0.2326, 0.2712, 0.2637,
        0.2323, 0.3333, 0.2714, 0.2991, 0.2747, 0.2515, 0.2394, 0.2709, 0.

       device='cuda:0', requires_grad=True))
('layer4.0.conv2.weight', Parameter containing:
tensor([[[[ 1.6218e-04, -1.4720e-02, -1.7000e-02],
          [-1.2850e-02, -3.3085e-02, -3.6656e-02],
          [ 2.7812e-02,  1.7691e-02, -1.8369e-02]],

         [[ 1.0528e-02,  3.1379e-02,  2.4801e-02],
          [-1.2698e-02, -2.9453e-02, -1.1834e-02],
          [-9.4094e-03, -8.9462e-03, -3.1349e-02]],

         [[-7.8447e-03, -2.9256e-02,  5.3590e-03],
          [-1.3791e-02, -1.1116e-02,  5.0388e-03],
          [-2.4919e-03,  7.3514e-03,  5.4013e-03]],

         ...,

         [[-1.0276e-03, -1.0275e-02, -2.9986e-02],
          [-3.8465e-03,  1.9549e-03, -1.6291e-02],
          [-1.8100e-03,  8.3778e-03, -8.5481e-03]],

         [[-1.8196e-02, -1.3533e-02, -1.7457e-02],
          [ 2.2457e-02,  5.7402e-02,  1.9325e-02],
          [-2.4977e-02, -3.2113e-02, -8.1780e-03]],

         [[ 3.6550e-03,  4.9358e-03, -5.7597e-03],
          [-1.6875e-02,  1.3999e-04,  3.7629e-04],
          [-2.6

('layer4.0.downsample.0.weight', Parameter containing:
tensor([[[[ 0.0057]],

         [[ 0.0020]],

         [[ 0.0167]],

         ...,

         [[ 0.0085]],

         [[-0.0274]],

         [[ 0.0097]]],


        [[[-0.0271]],

         [[-0.0157]],

         [[ 0.0543]],

         ...,

         [[-0.0206]],

         [[-0.0308]],

         [[ 0.0013]]],


        [[[-0.0523]],

         [[-0.0353]],

         [[ 0.0394]],

         ...,

         [[-0.0382]],

         [[-0.0264]],

         [[-0.0443]]],


        ...,


        [[[ 0.0494]],

         [[ 0.0436]],

         [[ 0.0103]],

         ...,

         [[ 0.0072]],

         [[ 0.0014]],

         [[-0.0669]]],


        [[[ 0.0533]],

         [[-0.0148]],

         [[-0.0480]],

         ...,

         [[ 0.0055]],

         [[ 0.0429]],

         [[ 0.0129]]],


        [[[ 0.0300]],

         [[-0.0092]],

         [[ 0.0090]],

         ...,

         [[ 0.0101]],

         [[-0.0111]],

         [[-0.0080]]]], d

('layer4.1.bn1.bias', Parameter containing:
tensor([-0.1668, -0.3019, -0.2187, -0.2917, -0.1971, -0.2325, -0.1869, -0.1857,
        -0.2474, -0.1629, -0.2448, -0.2508, -0.1895, -0.2651, -0.3250, -0.3811,
        -0.2953, -0.4963, -0.0294, -0.2724, -0.2007, -0.2220, -0.2945, -0.2579,
        -0.5152, -0.1994, -0.5016, -0.2736, -0.4528, -0.3968, -0.2281, -0.1772,
        -0.1293, -0.2655, -0.3252, -0.3232, -0.3337, -0.1901, -0.0692, -0.2196,
        -0.2132, -0.2565, -0.1646, -0.1567, -0.2087, -0.2178, -0.2480, -0.2767,
        -0.3071, -0.1988, -0.1985, -0.0235, -0.2458, -0.4156, -0.1660, -0.1923,
        -0.3328, -0.1481, -0.3047, -0.2277, -0.3182, -0.2744, -0.1643, -0.3365,
        -0.4050, -0.2082, -0.0621, -0.2671, -0.2809, -0.2185, -0.2148, -0.4465,
        -0.3376, -0.3213, -0.2921, -0.1998, -0.3369, -0.2092, -0.2831, -0.2893,
        -0.1719, -0.2189, -0.4016, -0.2484, -0.2070, -0.3849, -0.3753, -0.5874,
        -0.1637, -0.1748, -0.2217, -0.5067, -0.2496, -0.2117, -0.4291, -0.19

('layer4.1.bn2.weight', Parameter containing:
tensor([1.8419, 1.8307, 1.7650, 1.8288, 1.9505, 1.8026, 1.9536, 2.2790, 1.7662,
        1.8902, 1.7768, 1.7749, 1.9055, 1.7328, 1.8762, 1.8211, 1.7967, 2.3428,
        1.7985, 1.7271, 1.7915, 1.9512, 1.8928, 1.9017, 1.8784, 1.9809, 1.8569,
        1.7830, 1.8911, 1.8859, 1.7764, 1.9832, 1.8389, 1.7616, 1.8728, 1.8753,
        1.9008, 1.8209, 1.7039, 1.7377, 1.7786, 1.6944, 1.7829, 1.7815, 1.7594,
        1.8428, 1.9238, 2.0871, 1.8980, 1.8413, 1.8471, 1.8584, 1.7640, 1.8453,
        1.7606, 1.9504, 1.9620, 1.8755, 1.9424, 1.8731, 1.8674, 1.9422, 1.8750,
        1.9208, 1.7464, 1.8558, 1.6539, 2.0660, 2.0298, 1.9174, 1.8972, 1.7589,
        1.7551, 1.9560, 1.7909, 1.7971, 1.7851, 1.7733, 1.8061, 1.7949, 1.8169,
        1.8089, 1.8641, 2.1542, 1.7739, 1.7913, 1.8022, 1.7155, 1.7679, 1.7704,
        1.6266, 1.8645, 1.9076, 1.8576, 1.6924, 1.8020, 1.7100, 1.7713, 1.8572,
        1.7103, 2.0664, 1.9054, 1.9422, 1.8078, 1.7412, 1.6061, 1.9105, 1.

In [14]:
%%script false
found = False
for child in model.named_parameters():  
    if child[0] == 'fc.weight':
        found = True
    child[1].requires_grad = found

In [15]:
[i[0] for i in model.named_parameters() if i[1].requires_grad]

['fc.weight', 'fc.bias']

In [16]:
%%script false

# default - Validation: metrics  0.6033333333333334 loss:  0.6769222617149353

validate_vid_bf(model, x_val_vid, y_train, log_loss_sigmoid, accuracy_sigmoid_mean, device, 2, face_extractor, 
                fast_mtcnn=fast_mtcnn,
                print_results=True, 
                reverse=False)

Couldn't find program: 'false'


In [17]:
%%script false

#0.37783308817654526 center crop
#0.3752234482172356  256
#0.40455125311689993 224
#0.44303366757688983 299

validate_img(model, x_val, y_train, loss, accuracy_b, device, 10, 
             print_results=True, 
             inference=False,
             checkpoint=None,
             reverse=False
            )

Couldn't find program: 'false'


In [18]:
0.37011126635215613 * 100 / 83, 0.37011126635215613 + 0.08228

(0.4459171883760917, 0.45239126635215615)

In [19]:
#%%script false
# 38% / 47% / 57% /58.5% / 61% - 0.656% (0.99344%) - video
# 83% undersampling / 50% oversampling / 89.8% original dataset / 0.08228

metric, lost = train_img(model, loss, optimizer, scheduler, x_train, x_val, y_train, accuracy_b, device, [log_loss_sigmoid, accuracy_sigmoid_mean], face_extractor, 
                          epochs=20, 
                       batch_size=batch_size, 
                       del_net=False, 
                       useInference=False, 
                       inference=False, 
                       useScheduler=True,
                       checkpoint=[0.37, 0.99],
                         reverse=False
                      )


Validation: metrics  0.5880151423900886 loss  0.6745712998861908
Train: metrics  0.6211410142348754 loss  0.6409693282163864
Expected LB value: 0.7568512998861908 0.8127365058869768
Epoch: 1



Validation: metrics  0.59012303191947 loss  0.6703475730521704
Train: metrics  0.647942615658363 loss  0.6198630925599367
Expected LB value: 0.7526275730521704 0.807647678376109
Epoch: 2


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "d:\users\user\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-19-5cad7254c1cf>", line 13, in <module>
    reverse=False
  File "C:\Users\user\Desktop\Projects\dfdc-kaggle-solution\pipeline\model_methods.py", line 177, in train_img
    preds = net(X_batch.to(device))
  File "d:\users\user\appdata\local\programs\python\python36\lib\site-packages\torch\nn\modules\module.py", line 493, in __call__
    result = self.forward(*input, **kwargs)
  File "C:\Users\user\AppData\Roaming\Python\Python36\site-packages\torchvision\models\resnet.py", line 204, in forward
    x = self.layer4(x)
  File "d:\users\user\appdata\local\programs\python\python36\lib\site-packages\torch\nn\modules\module.py", line 493, in __call__
    result = self.forward(*input, **kwargs)
  File "d:\users\user\appdata\local\programs\python\pyt

KeyboardInterrupt: 

In [ ]:
from operator import itemgetter
min_lost = max(enumerate(lost), key=itemgetter(1))[0] 
min_metrics = min(enumerate(metric), key=itemgetter(1))[0] 
print("Loss optim:", lost[min_lost], min_lost + 1)
print("Metrics optim:", metric[min_metrics], min_metrics + 1)

In [ ]:
plt.plot(metric, label='model')
plt.legend()
plt.title('Validation metrics')

In [ ]:
plt.plot(lost, label='model')
plt.legend()
plt.title('Validation Loss')

In [ ]:
'''model = torch.load(PATH)
model = model.to(device)
model.eval()'''

In [ ]:
validate_img(model, x_val, y_train, loss, log_loss, device, batch_size, print_results=True, inference=nn.Softmax(dim=1))

In [ ]:
validate_img(model, x_val, y_train, loss, log_loss, device, batch_size, show_results=True, inference=nn.Softmax(dim=1))

In [ ]:
validate_img(model, x_val, y_train, loss, log_loss, device, batch_size, show_graphic=True, inference=nn.Softmax(dim=1))

torch.save(model.state_dict(), 'model.pth')